# Cycle Indicators - Apple Stock 2023-2025

Testing Hilbert Transform cycle indicators.

In [1]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
from pathlib import Path
sys.path.insert(0, str(Path('../..').resolve()))

from indicators.cycle import calculate_ht_sine, calculate_ht_trendmode

In [2]:
# Fetch Apple data
df = yf.download('AAPL', start='2023-01-01', end='2025-10-01', auto_adjust=True, progress=False)

# Handle MultiIndex columns from yfinance
if df.columns.nlevels == 2:
    df.columns = df.columns.get_level_values(0)

df.columns = df.columns.str.lower()
df = df.reset_index()
df.columns = df.columns.str.lower()

print(f"Data shape: {df.shape}")
df.head()

Data shape: (688, 6)


Price,date,close,high,low,open,volume
0,2023-01-03,123.330658,129.079575,122.443173,128.468202,112117500
1,2023-01-04,124.602707,126.870724,123.340509,125.125335,89113600
2,2023-01-05,123.281342,125.993097,123.024963,125.361998,80962700
3,2023-01-06,127.817352,128.478033,123.153137,124.257564,87754700
4,2023-01-09,128.339966,131.554638,128.083587,128.655523,70790800


## 1. HT_SINE - Hilbert Transform Sine Wave

Detects market cycles using sine wave oscillator. Crossovers between sine and leadsine signal cycle reversals.

In [3]:
sine, leadsine = calculate_ht_sine(df)
df['sine'] = sine
df['leadsine'] = leadsine

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=('Price', 'Sine Wave Oscillator'))

fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close', line=dict(color='black')), row=1, col=1)

fig.add_trace(go.Scatter(x=df['date'], y=df['sine'], name='Sine', line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=df['date'], y=df['leadsine'], name='Lead Sine', 
                        line=dict(color='red', dash='dash')), row=2, col=1)
fig.add_hline(y=0, line_dash="dot", line_color="gray", row=2, col=1)

fig.update_layout(height=600, title_text="HT_SINE - Cycle Detection")
fig.show()

print("Sine wave detects market cycles.")
print("Crossovers between sine and leadsine can signal reversals.")

Sine wave detects market cycles.
Crossovers between sine and leadsine can signal reversals.


## 2. HT_TRENDMODE - Trend vs Cycle Mode

Identifies whether market is in trending mode (1) or cycling mode (0). Helps adapt strategy to market conditions.

In [4]:
trendmode = calculate_ht_trendmode(df, threshold=0.5)
df['trendmode'] = trendmode

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=('Price', 'Trend Mode (1=Trend, 0=Cycle)'))

fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close', line=dict(color='black')), row=1, col=1)

# Color price based on mode
trend_periods = df[df['trendmode'] == 1]
cycle_periods = df[df['trendmode'] == 0]

fig.add_trace(go.Scatter(x=trend_periods['date'], y=trend_periods['close'], mode='markers',
                        name='Trend Mode', marker=dict(color='blue', size=3)), row=1, col=1)
fig.add_trace(go.Scatter(x=cycle_periods['date'], y=cycle_periods['close'], mode='markers',
                        name='Cycle Mode', marker=dict(color='red', size=3)), row=1, col=1)

fig.add_trace(go.Scatter(x=df['date'], y=df['trendmode'], name='Mode',
                        fill='tozeroy', line=dict(color='purple')), row=2, col=1)

fig.update_layout(height=600, title_text="HT_TRENDMODE - Market State Detection")
fig.show()

print(f"Trend periods: {(df['trendmode'] == 1).sum()} days")
print(f"Cycle periods: {(df['trendmode'] == 0).sum()} days")

Trend periods: 541 days
Cycle periods: 147 days


## Combined View - All Cycle Indicators

In [5]:
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=(
                        'Price with Trend/Cycle Coloring',
                        'Sine Wave Oscillator',
                        'Trend Mode'
                    ))

# Row 1: Price
fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close', 
                        line=dict(color='black', width=1)), row=1, col=1)
trend_periods = df[df['trendmode'] == 1]
cycle_periods = df[df['trendmode'] == 0]
fig.add_trace(go.Scatter(x=trend_periods['date'], y=trend_periods['close'], mode='markers',
                        name='Trend', marker=dict(color='blue', size=4)), row=1, col=1)
fig.add_trace(go.Scatter(x=cycle_periods['date'], y=cycle_periods['close'], mode='markers',
                        name='Cycle', marker=dict(color='red', size=4)), row=1, col=1)

# Row 2: Sine waves
fig.add_trace(go.Scatter(x=df['date'], y=df['sine'], name='Sine',
                        line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=df['date'], y=df['leadsine'], name='Lead Sine',
                        line=dict(color='red', dash='dash')), row=2, col=1)
fig.add_hline(y=0, line_dash="dot", line_color="gray", row=2, col=1)

# Row 3: Trend mode
fig.add_trace(go.Scatter(x=df['date'], y=df['trendmode'], name='Trend Mode',
                        fill='tozeroy', line=dict(color='purple')), row=3, col=1)

fig.update_layout(height=900, title_text="Cycle Indicators - Complete Overview")
fig.show()

print("\n=== Cycle Analysis Summary ===")
print(f"Total days: {len(df)}")
print(f"Trend mode: {(df['trendmode'] == 1).sum()} days ({(df['trendmode'] == 1).sum() / len(df) * 100:.1f}%)")
print(f"Cycle mode: {(df['trendmode'] == 0).sum()} days ({(df['trendmode'] == 0).sum() / len(df) * 100:.1f}%)")


=== Cycle Analysis Summary ===
Total days: 688
Trend mode: 541 days (78.6%)
Cycle mode: 147 days (21.4%)
